# GeoProj - Corrección de Distorsión Geométrica en Imágenes

Este notebook implementa el método de **Blind Geometric Distortion Correction** usando Deep Learning.

**Paper:** Li et al., CVPR 2019  
**Repositorio:** https://github.com/xiaoyu258/GeoProj

---

## 🎯 Tipos de Distorsión Detectados
- **Barrel** (barril)
- **Pincushion** (cojín)
- **Rotation** (rotación)
- **Shear** (cizallamiento)
- **Projective** (proyectiva)
- **Wave** (onda)


## 1️⃣ Configuración Inicial - Verificar GPU


In [ ]:
# Verificar GPU
!nvidia-smi


In [ ]:
# Instalar dependencias
!pip install torch torchvision --quiet
!pip install scikit-image scipy numba matplotlib pillow --quiet

print("✓ Dependencias instaladas")


## 2️⃣ Clonar Repositorio


In [ ]:
import os

# Clonar el repositorio (cambiar URL por tu fork si lo has modificado)
REPO_URL = "https://github.com/xiaoyu258/GeoProj.git"  # Cambiar por tu repo

if not os.path.exists('GeoProj'):
    !git clone {REPO_URL}
    print("✓ Repositorio clonado")
else:
    print("✓ Repositorio ya existe")

# Cambiar al directorio del proyecto
%cd GeoProj


## 3️⃣ Descargar Modelos Pre-entrenados

**IMPORTANTE:** Descarga los modelos manualmente desde:  
https://drive.google.com/open?id=1Tdi92IMA-rrX2ozdUMvfiN0jCZY7wIp_

Luego súbelos usando la celda siguiente.


In [ ]:
from google.colab import files
import os

os.makedirs('models', exist_ok=True)

print("Sube los 3 archivos del modelo (.pkl):")
print("  - model_en.pkl")
print("  - model_de.pkl")
print("  - model_class.pkl")
print("\nHaz clic en 'Choose Files' y selecciona los archivos")

uploaded = files.upload()

# Mover a carpeta models
for filename in uploaded.keys():
    !mv {filename} models/
    print(f"✓ {filename} movido a models/")

# Verificar
print("\nArchivos en models/:")
!ls -lh models/


## 4️⃣ Subir Imagen para Procesar


In [ ]:
from google.colab import files
from IPython.display import Image as IPImage, display
import os

print("Sube tu imagen distorsionada:")
uploaded = files.upload()

# Obtener nombre del archivo subido
input_image = list(uploaded.keys())[0]
print(f"\n✓ Imagen subida: {input_image}")

# Mostrar imagen
display(IPImage(filename=input_image, width=400))


## 5️⃣ Procesar Imagen (Detección + Flow)


In [ ]:
# Ejecutar inferencia
!python inference.py --input_image {input_image} --output_dir results --model_dir models

print("\n" + "="*60)
print("✓ Procesamiento completado")
print("="*60)


In [ ]:
# Visualizar el flujo óptico detectado
from IPython.display import Image as IPImage, display
import os

base_name = os.path.splitext(input_image)[0]
flow_viz_path = f'results/{base_name}_flow_viz.png'

if os.path.exists(flow_viz_path):
    print("Visualización del Flujo Óptico:")
    display(IPImage(filename=flow_viz_path, width=800))
else:
    print("No se encontró la visualización del flujo")


## 6️⃣ Aplicar Corrección (Rectificación)

**Nota:** Esto requiere GPU con CUDA. Si no funciona en Colab, el flujo ya está calculado.


In [ ]:
import os

base_name = os.path.splitext(input_image)[0]
resized_path = f'results/{base_name}_resized.jpg'
flow_path = f'results/{base_name}_flow.npy'

try:
    # Intentar rectificación
    !python rectify.py --img_path {resized_path} --flow_path {flow_path} --output_dir results
    rectification_success = True
except Exception as e:
    print(f"Error en rectificación: {e}")
    print("\nLa rectificación requiere CUDA. Puedes descargar el flujo y procesarlo localmente.")
    rectification_success = False


## 7️⃣ Ver Resultados


In [ ]:
from IPython.display import Image as IPImage, display
import os

base_name = os.path.splitext(input_image)[0]

# Mostrar comparación si existe
comparison_path = f'results/{base_name}_comparison.png'
if os.path.exists(comparison_path):
    print("Comparación Antes/Después:")
    display(IPImage(filename=comparison_path, width=800))
else:
    print("Comparación no disponible")
    
    # Mostrar imagen corregida si existe
    corrected_path = f'results/{base_name}_corrected.png'
    if os.path.exists(corrected_path):
        print("\nImagen Corregida:")
        display(IPImage(filename=corrected_path, width=400))


## 8️⃣ Descargar Resultados


In [ ]:
# Comprimir resultados
!zip -r results.zip results/

# Descargar
from google.colab import files
files.download('results.zip')

print("✓ Resultados comprimidos y listos para descargar")


---

## 📝 Notas Importantes

1. **Tamaño de imagen:** El modelo funciona con imágenes de 256x256. Las imágenes se redimensionan automáticamente.

2. **GPU:** Se recomienda usar GPU para mejor rendimiento. En Colab: `Runtime` > `Change runtime type` > `GPU`

3. **CUDA para rectificación:** La etapa de rectificación usa numba-cuda y requiere GPU.

4. **Archivos generados:**
   - `*_flow.npy`: Flujo óptico calculado
   - `*_flow_viz.png`: Visualización del flujo
   - `*_resized.jpg`: Imagen redimensionada
   - `*_corrected.png`: Imagen corregida
   - `*_comparison.png`: Comparación antes/después

---

## 📚 Referencias

**Paper:** Li, X., Zhang, B., Sander, P. V., & Liao, J. (2019). *Blind Geometric Distortion Correction on Images Through Deep Learning.* CVPR 2019.

**ArXiv:** https://arxiv.org/abs/1909.03459  
**GitHub:** https://github.com/xiaoyu258/GeoProj
